In [1]:
%load_ext autoreload 
# saving changes to code reloads them in the environment
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import sys
import funcy
from __future__ import division
import numpy as np
import scipy.optimize as spo
import pandas as pd
from collections import defaultdict
from code.gradient_descent import _gradient_descent, gradient_descent, SGD, g_error, b_error 
from code.helpers import f_gauss, d_gauss, f_bowl, d_bowl, get_lr
from code.basis_func import create_basis, create_cos_basis, BasisSearch, plotter
from code.constants import *
import pdb
import random
import pylab as pl

import os
from sklearn.linear_model import *
import seaborn as sns

In [2]:
# X is an array of N data points (one dimensional for now), that is, NX1
# Y is a Nx1 column vector of data values

def getData(name):
    data = pl.loadtxt(os.path.join('hw1code/P3', name))
    # Returns column matrices
    X = data[0:1].T
    Y = data[1:2].T
    return X, Y

AX, AY = getData('regressA_train.txt')
BX, BY =getData('regressB_train.txt')
VX, VY = getData('regress_validate.txt')

In [3]:
%%latex
Weights for Ridge: from bishop
$ w = (λI + Φ_{T}Φ_{−1}) Φ_{T}y$

<IPython.core.display.Latex object>

In [ ]:
axt = create_basis(AX, M=9)[0]

In [ ]:
cdata = pl.loadtxt('hw1code/P2/curvefittingp2.txt')

cX = np.array([cdata[0]]).T
cY = cdata[1,:]


In [ ]:
pd.DataFrame({'us': np.squeeze(ridge_weights(AX, AY, lab)),
              'clf': np.squeeze(LinearRegression().(AX, AY))}).corr()

In [ ]:
cY.shape

# 3.1 Implement Ridge

In [26]:
def ridge_weights(x, y, M=9, l=.5):
    axt = create_basis(x, M=M)[0]
    
    lh = np.linalg.inv(np.eye(axt.shape[1])*l+ axt.T.dot(axt))
    return np.squeeze(lh.dot(axt.T.dot(y)))

def ridge_loss(x, y, w=None, M=9, l=.5):
    axt = create_basis(x, M=M)[0]
    #print axt.shape
    if w is None:
        lh = np.linalg.inv(np.eye(axt.shape[1])*l+ axt.T.dot(axt))
        w = np.squeeze(lh.dot(axt.T.dot(y)))
    yhat = axt.dot(w)
    err = (yhat - np.squeeze(y))
    assert err.shape == yhat.shape
    mse = np.mean(err ** 2)
    return mse

def clf_w(Xtr, ytr, M=9, clf=Ridge()):
    return np.squeeze(clf.fit(create_basis(Xtr, M=M)[0], ytr).coef_)
    #clf.predict(create_basis(Xte, M=M)[0])

In [16]:
dev = np.sum(np.abs(clf_w(AX, AY, clf=LinearRegression(fit_intercept=False)) - ridge_weights(AX, AY, l=0)))
assert np.round(dev, 3) == 0


In [6]:
lspace = [0, .1, .5, 1, 3,5,10, 12, 100, 200, 1000, 10000]
mspace  = range(1, 6)
def loss_map(cX, cY, lspace=lspace, mspace=mspace):
    return pd.DataFrame({l: pd.Series({m: ridge_loss(cX, cY,M=m, l=l)
                                for m in mspace})
                for l in lspace}).round(3).rename_axis('M').rename_axis('L', 1)

In [7]:
def oos_loss(Xtr, ytr, Xte, yte, **kwargs):
    w = ridge_weights(Xtr, ytr, **kwargs)
    return ridge_loss(Xte, yte, w=w, **kwargs)
def iss_loss(Xtr, ytr, Xte, yte, **kwargs):
    return ridge_loss(Xtr, ytr, **kwargs)

In [37]:
AX5 = create_basis(AX, M=5)[0]
yhat = LinearRegression(fit_intercept=False).fit(AX5, AY).predict(AX5)
desired_mse = np.mean((yhat - AY)**2)
assert np.round(ridge_loss(AX, AY, M=5, l=0) -  desired_mse, 4) == 0

In [29]:
def oos_loss_map(Xtr, ytr, Xte, yte, lspace=lspace, mspace=mspace):
    return pd.DataFrame({l: pd.Series({m: oos_loss(Xtr, ytr, Xte, yte, M=m, l=l)
                                for m in mspace})
                for l in lspace}).round(3).rename_axis('M').rename_axis('L', 1)

# 3.2 Train, Test, Validate for different `L` and `M` 

In [ ]:
ridge_loss(BX, BY, w=ridge_weights(AX, AY))

In [ ]:
ridge_loss(BX, BY, w=ridge_weights(AX, AY))

In [ ]:
AX.shape

In [ ]:
BX.shape

In [ ]:
VX.shape

In [ ]:
ridge_loss(AX, AY, w=ridge_weights(BX, BY))

In [ ]:
clf_loss(AX, AY, BX, BY)

In [ ]:
ridge_weights(AX, AY)

In [32]:
def oos_loss_map(Xtr, ytr, Xte, yte, lspace=lspace, mspace=mspace):
    return pd.DataFrame({l: pd.Series({m: oos_loss(Xtr, ytr, Xte, yte, M=m, l=l)
                                for m in mspace})
                for l in lspace}).round(3).rename_axis('M').rename_axis('L', 1)

In [33]:
def iss_loss_map(Xtr, ytr, Xte=None, yte=None, lspace=lspace, mspace=mspace):
    return pd.DataFrame({l: pd.Series({m: ridge_loss(Xtr, ytr, M=m, l=l)
                                for m in mspace})
                for l in lspace}).round(3).rename_axis('M').rename_axis('L', 1)

In [77]:
loss_av = oos_loss_map(AX, AY, VX, VY)
loss_ab = oos_loss_map(AX, AY, BX, BY)
loss_a = iss_loss_map(AX, AY)
loss_b = iss_loss_map(BX, BY)
loss_bv = oos_loss_map(BX, BY, VX, VY)
loss_ba = oos_loss_map(BX, BY, AX, AY)


In [35]:
loss_a

L,0.0,0.1,0.5,1.0,3.0,5.0,10.0,12.0,100.0,200.0,1000.0,10000.0
M,,,,,,,,,,,,
1,0.076,0.076,0.078,0.082,0.120,0.175,0.333,0.397,1.825,2.318,2.927,3.107
2,0.067,0.067,0.071,0.078,0.128,0.185,0.328,0.383,1.796,2.307,2.927,3.107
3,0.067,0.068,0.073,0.086,0.161,0.231,0.355,0.389,0.747,0.908,1.491,2.655
4,0.046,0.047,0.050,0.059,0.117,0.180,0.308,0.347,0.764,0.867,1.395,2.650
5,0.044,0.045,0.053,0.065,0.110,0.154,0.259,0.297,0.829,0.936,1.058,1.390


In [36]:
loss_av

L,0.0,0.1,0.5,1.0,3.0,5.0,10.0,12.0,100.0,200.0,1000.0,10000.0
M,,,,,,,,,,,,
1,0.132,0.134,0.142,0.155,0.225,0.308,0.521,0.603,2.191,2.687,3.284,3.457
2,0.107,0.110,0.126,0.147,0.239,0.325,0.518,0.588,2.137,2.660,3.282,3.457
3,0.108,0.116,0.151,0.198,0.374,0.512,0.735,0.795,1.343,1.535,2.161,3.136
4,0.163,0.167,0.189,0.221,0.355,0.472,0.686,0.749,1.378,1.522,2.074,3.161
5,0.188,0.180,0.173,0.181,0.251,0.329,0.518,0.586,1.564,1.768,1.979,2.253


In [39]:
loss_b

L,0.0,0.1,0.5,1.0,3.0,5.0,10.0,12.0,100.0,200.0,1000.0,10000.0
M,,,,,,,,,,,,
1,1.440,1.440,1.445,1.460,1.573,1.720,2.081,2.207,3.739,4.003,4.254,4.317
2,1.218,1.218,1.221,1.227,1.269,1.325,1.486,1.552,2.907,3.367,4.048,4.293
3,0.983,0.984,1.003,1.041,1.198,1.317,1.518,1.580,2.843,3.352,4.046,4.290
4,0.708,0.718,0.802,0.895,1.134,1.282,1.510,1.572,2.348,2.701,3.452,4.015
5,0.078,0.106,0.273,0.396,0.627,0.784,1.097,1.199,2.388,2.667,3.383,3.993


In [38]:
loss_bv

L,0.0,0.1,0.5,1.0,3.0,5.0,10.0,12.0,100.0,200.0,1000.0,10000.0
M,,,,,,,,,,,,
1,1.600,1.582,1.520,1.459,1.339,1.328,1.463,1.535,2.849,3.127,3.400,3.470
2,1.751,1.753,1.760,1.768,1.795,1.824,1.918,1.960,2.808,3.031,3.345,3.462
3,1.377,1.339,1.278,1.304,1.591,1.801,2.016,2.045,2.495,2.918,3.453,3.487
4,5.918,4.428,2.505,2.059,2.230,2.595,3.200,3.345,3.166,2.951,3.337,3.488
5,28.429,22.356,14.001,10.521,5.367,3.336,1.796,1.698,4.036,3.705,3.111,3.982


In [40]:
ridge_weights(AX,AY,M=5, l=0)

array([ 0.60503682,  1.1303877 ,  0.30816594, -0.13840606, -0.04149159,
        0.01327405])

In [41]:
ridge_weights(AX,AY,M=5, l=1000)

array([ 0.00704325,  0.00893167,  0.01027522,  0.01301723,  0.02851861,
        0.02052394])

In [42]:
res = loss_av.stack().sort_values()

In [48]:
def score(AX, AY, VX, VY, BX, BY):
    loss_av = oos_loss_map(AX, AY, VX, VY)
    res = loss_av.stack().sort_values()
    opt_m, opt_l = res.idxmin()
    w_opt = ridge_weights(AX, AY, M=opt_m, l=opt_l)
    return pd.Series({'Test MSE': ridge_loss(BX, BY, w=w_opt, M=opt_m),
                      'Validation MSE': ridge_loss(VX, VY, w=w_opt, M=opt_m),
                      'Train MSE': ridge_loss(AX, AY, w=w_opt, M=opt_m),
                     'Optimal M': opt_m, 'Optimal L': opt_l}).round(2)


In [49]:
a = score(AX, AY, VX, VY, BX, BY)

In [50]:
b = score(BX, BY, VX, VY, AX, AY)

In [66]:
pd.DataFrame({'Train on A': a, 'Train on B':b})#.to_latex()

,Train on A,Train on B
Optimal L,0.00,0.50
Optimal M,2.00,3.00
Test MSE,2.58,3.16
Train MSE,0.07,1.00
Validation MSE,0.11,1.28


In [ ]:
opt_m, opt_l = res.idxmin()

In [ ]:
res.min()

In [ ]:
w_opt = ridge_weights(AX, AY, M=opt_m, l=opt_l)

In [ ]:
a_tr = pd.Series({'test': ridge_loss(BX, BY, w=w_opt, M=2),
 'validation': ridge_loss(VX, VY, w=w_opt, M=2),
 'train': ridge_loss(AX, AY, w=w_opt, M=2)}).round(2)

In [ ]:
from ggplot import *

In [ ]:
ridge_loss(AX, AY, l=0, M=5)

In [ ]:
ridge_loss(AX, AY, l=100, M=5)

In [ ]:
loss_a.stack().sort_values()#.rename('Test MSE')

In [90]:
pl_data  = pd.DataFrame({'Train MSE': loss_a.stack(), 
                         'Validation MSE': loss_av.stack(),
                        'Test MSE': loss_ab.stack()}).reset_index()

In [110]:
plb  = pd.DataFrame({'Train MSE': loss_b.stack(), 
                         'Validation MSE': loss_bv.stack(),
                        'Test MSE': loss_ba.stack()}).reset_index()

In [118]:
plb2 = plb[(plb.L.isin([0, .5, 5])) & (plb.M.isin([1,3]))]

In [119]:
print plb2.set_index(['M', 'L']).round(2)[[ u'Train MSE', u'Validation MSE', u'Test MSE']].to_latex()

\begin{tabular}{llrrr}
\toprule
  &     &  Train MSE &  Validation MSE &  Test MSE \\
M & L &            &                 &           \\
\midrule
1 & 0.0 &       1.44 &            1.60 &      1.82 \\
  & 0.5 &       1.44 &            1.52 &      1.71 \\
  & 5.0 &       1.72 &            1.33 &      1.36 \\
3 & 0.0 &       0.98 &            1.38 &      3.76 \\
  & 0.5 &       1.00 &            1.28 &      3.16 \\
  & 5.0 &       1.32 &            1.80 &      2.58 \\
\bottomrule
\end{tabular}



In [121]:
#plb.sort_values('Test MSE')

In [98]:
pl_data.corr()

,M,L,Test MSE,Train MSE,Validation MSE
M,1.000000,0.000000,0.117767,-0.219339,-0.114805
L,0.000000,1.000000,0.547087,0.665889,0.654658
Test MSE,0.117767,0.547087,1.000000,0.792975,0.887624
Train MSE,-0.219339,0.665889,0.792975,1.000000,0.982422
Validation MSE,-0.114805,0.654658,0.887624,0.982422,1.000000


In [94]:
pl_data[pl_data.]

,M,L,Test MSE,Train MSE,Validation MSE
0,1,0.0,2.510,0.076,0.132
1,1,0.1,2.513,0.076,0.134
2,1,0.5,2.527,0.078,0.142
3,1,1.0,2.546,0.082,0.155
4,1,3.0,2.623,0.120,0.225
5,1,5.0,2.700,0.175,0.308
6,1,10.0,2.866,0.333,0.521
7,1,12.0,2.922,0.397,0.603
8,1,100.0,3.736,1.825,2.191
9,1,200.0,3.960,2.318,2.687


In [86]:
tab =  pl_data.set_index("M").sort_index().dropna().round(2)

In [88]:
tab.sort_values('Validation MSE')

,L,Test MSE,Train MSE,Validation MSE
M,,,,
2,0.0,2.58,0.07,0.11
3,0.0,2.58,0.07,0.11
1,0.0,2.51,0.08,0.13
4,0.0,2.43,0.05,0.16
5,0.0,2.32,0.04,0.19
1,5.0,2.70,0.18,0.31
2,5.0,2.62,0.18,0.32
5,5.0,2.98,0.15,0.33
4,5.0,2.81,0.18,0.47


In [ ]:
(
    ggplot(aes(x='M', y='Validation', colour='L'),
           data=pl_data.assign(L=lambda x: x.L.astype(str)))+
    geom_line() + 
    ylab('MSE') + 
    xlab('M (polynomial order)') +
    ggtitle('Validation MSE vs. M')
    )

In [ ]:
p

In [ ]:
loss_b.stack().to_frame(name='MSE').reset_index().corr().round(2)

In [ ]:
loss_av#.stack().idxmin()

In [ ]:
loss_bv.stack().idxmin()

In [ ]:
loss_av.stack().max()

In [ ]:
#loss_bv.stack().sort_values()

In [ ]:
#sns.heatmap(np.clip(loss_bv, 0, loss_av.stack().max()))

In [ ]:
loss_av.stack().sort_values()#.idxmin()

In [ ]:
ax = sns.heatmap(loss_av.round(2), xticklabels=2)
ax.set_title('Validation MSE from training on Dataset A')

In [ ]:
ax = sns.heatmap(loss_av.round(2), xticklabels=2)
ax.set_title('Validation MSE from training on Dataset A')

In [ ]:
ax = sns.heatmap(np.clip(loss_bv, 0, loss_av.stack().max()),
                 xticklabels=2)
ax.set_title('Validation MSE from training on Dataset B')

In [ ]:
#Ridge()

In [ ]:
sns.heatmap(np.clip(loss_b, 0, loss_av.stack().max()))

loss_b.stack().sort_values()

In [ ]:
loss_b.median().plot()

In [ ]:
loss_b.median(1).plot()

In [ ]:
loss_b.mean().plot()

In [ ]:
loss_b.mean(1).plot()

In [ ]:
pd.DataFrame({'x': np.squeeze(AX), 'y': np.squeeze(AY)}).plot.scatter(x='x', y='y')

In [ ]:
pd.DataFrame({'x': np.squeeze(BX), 'y': np.squeeze(BY)}).plot.scatter(x='x', y='y')